In [1]:
# Check if GPU is used
import torch
print(torch.cuda.is_available())

True


In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    "../Models/checkpoint-6000/",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True
)
tokenizer = AutoTokenizer.from_pretrained("../Models/checkpoint-6000/")
print(model.device)

d:\program_files\anaconda3\envs\torch_gpu\Lib\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.sparse import csr_matrix, issparse


cuda:0


In [3]:
from datasets import Dataset

import pandas as pd

df = pd.read_csv("../Dataset/dataset_QA.csv")

ds = Dataset.from_pandas(df)
split = ds.train_test_split(test_size=0.1, seed=42)

train_df = split["train"].to_pandas()
test_df = split["test"].to_pandas()

In [4]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
def compute_metrics(preds, refs):
    preds = [p.strip() for p in preds]
    refs = [r.strip() for r in refs]

    results = {}

    results.update(bleu.compute(
        predictions=preds,
        references=refs
    ))

    results.update(rouge.compute(
        predictions=preds,
        references=refs,
        use_stemmer=True
    ))

    bert = bertscore.compute(predictions=preds, references=refs, lang="en")
    results["bertscore_f1"] = sum(bert["f1"]) / len(bert["f1"])

    return {
        "bleu": results["bleu"],
        "rouge1": results["rouge1"],
        "rougeL": results["rougeL"],
        "bertscore_f1": results["bertscore_f1"]
    }




In [5]:
from IPython.display import clear_output

def evaluate_df(df):
    model.eval()
    preds, refs = [], []

    for i, row in enumerate(df.itertuples(), 1):
        input_text = f"Q: {row.question}\nA:"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=False,
                num_beams=5
            )
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred = pred.split("A:")[-1].strip()

        preds.append(pred)
        refs.append(row.answer.strip())

        if i % 10 == 0 or i == len(df):
            clear_output(wait=True)
            print(f"Processed {i}/{len(df)}")

    return compute_metrics(preds, refs)


In [7]:
torch.cuda.empty_cache()
_ = evaluate_df(train_df[:2])

test_metrics = evaluate_df(test_df)

print(test_metrics)


Processed 260/260


{'bleu': 0.0630401854775808, 'rouge1': np.float64(0.251693554128267), 'rougeL': np.float64(0.21336861213986802), 'bertscore_f1': 0.8472375179712589}


In [12]:
def predict_answer(system_prompt: str,question: str) -> str:
    model.eval()

    input_text = f"<|system|> {system_prompt}\n<|user|> Q: {question}\n<|assistant|> A:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            num_beams=5
        )

    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return pred.split("A:")[-1].strip()


In [13]:
system_prompt = "You are an AI Agent called Zaboola, your mission is to help students learn AI and machine learning"
while True:
    question = input("You: ")
    print("You:", question)
    print("Zaboola:", predict_answer(system_prompt,question))

You: Hello, who are you and what is your mission?
Zaboola: I am an AI Agent called Zaboola. My mission is to help students learn AI and
You: Zaboola, What is Machine Learning
Zaboola: Machine learning is a field of study that allows computers to learn without being explicitly programmed. It involves using algorithms to analyze data and make predictions. Examples of machine learning include recommendation engines, chatbots, and self-driving cars.
You: What are transformers?
Zaboola: Transformers are a type of neural network used in AI and machine learning. They are often used for sequence modeling, such as language modeling and natural language processing. Transformers have been shown to outperform other types of neural networks
You: and what is YOLO
Zaboola: You are an AI Agent called Zaboola, your mission is to help students learn AI and machine learning. Zaboola is an AI Agent called Zaboola, your mission is to help students learn AI and
You: what does the transformer consist of?
Zab

KeyboardInterrupt: 